In [1]:
%cd ../src

e:\00_CODE\03_Master_Thesis\rdf-literal-preprocessing\src


In [2]:
import fire
import sys
import tqdm
import pandas as pd
import numpy as np
import kgbench as kg
from collections import Counter
import matplotlib.pyplot as pl
import random as rd
from operator import itemgetter
import torch
import torch.nn.functional as F
from utils import RDF_NUMBER_TYPES, get_relevant_relations, add_triple, get_p_types, ALL_LITERALS
from kgbench.load import Data
from typing import List, Sequence, Tuple

from kgbench import load, tic, toc, d
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from utils import URI_PREFIX
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
import datetime
from typing import List, Sequence, Tuple
from utils import URI_PREFIX
from preprocess.binning import delete_empty_bin_types

c:\Users\Noctris\.virtualenvs\rdf-literal-preprocessing-20b3_M0v\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from dataload import dmg777k
data = dmg777k()

loaded data dmg777k (55.89s).
pruned (9.622s).


# Alterating bins

In [28]:
num_bins = 3
relevent_relations = get_relevant_relations(
    data, relevant_types=RDF_NUMBER_TYPES)

for b in range(num_bins):
    o = (f'{URI_PREFIX}entity#binning{b+1}#1', f'{URI_PREFIX}datatype#bin')
    new_id = len(data.i2e)
    data.e2i[o] = new_id
    data.i2e.append(o)
    data.num_entities += 1

    o = (f'{URI_PREFIX}entity#binning{b}#2', f'{URI_PREFIX}datatype#bin')
    new_id = len(data.i2e)
    data.e2i[o] = new_id
    data.i2e.append(o)
    data.num_entities += 1

for r in relevent_relations:
    p = f'{URI_PREFIX}predicat#binning{r}#1'
    new_id = len(data.i2r)
    data.r2i[p] = new_id
    data.i2r.append(p)
    data.num_relations += 1
    p = f'{URI_PREFIX}predicat#binning{r}#2'
    new_id = len(data.i2r)
    data.r2i[p] = new_id
    data.i2r.append(p)
    data.num_relations += 1

for relation in relevent_relations:

    sub_df1 = encode_number_sublist(
        data.triples[data.triples[:, 1] == relation], data.i2e)
    sub_df2 = encode_number_sublist(
        data.triples[data.triples[:, 1] == relation], data.i2e)

    # numpy is used here since torch.histc was not working for some reason.
    sub_df1 = torch.cat(  # put bins and sub_df together
        (sub_df1, torch.from_numpy(  # get numpy solutions back
            np.digitize(  # assign for each value in sub_df the corresponding bin
                sub_df1[:, 1], np.histogram(  # calculate n bins based on values in sub_df
                    sub_df1[:, 1], num_bins*2)[1][:-1:2]
            )
        ).reshape(-1, 1)  # transfrom x tensor into (x,1) tensor to fit (x,2) shape of sub_df
        ), 1)
    sub_df2 = torch.cat(  # put bins and sub_df together
        (sub_df2, torch.from_numpy(  # get numpy solutions back
            np.digitize(  # assign for each value in sub_df the corresponding bin
                sub_df2[:, 1], np.histogram(  # calculate n bins based on values in sub_df
                    sub_df2[:, 1], num_bins*2)[1][1::2]
            )
        ).reshape(-1, 1)  # transfrom x tensor into (x,1) tensor to fit (x,2) shape of sub_df
        ), 1)

    object_mapping1 = np.vectorize(lambda t: data.e2i[(
        f'{URI_PREFIX}entity#binning{t}#1', f'{URI_PREFIX}datatype#bin')])

    predicat_mapping1 = np.vectorize(
        lambda t: data.r2i[f'{URI_PREFIX}predicat#binning{relation}#1'])

    sub_df1[:, 1] = torch.tensor([predicat_mapping1(sub_df1[:, 2])], dtype=torch.int32)
    sub_df1[:, 2] = torch.tensor([object_mapping1(sub_df1[:, 2])], dtype=torch.int32)
    data.triples = torch.cat((data.triples, sub_df1), 0)

    object_mapping2 = np.vectorize(lambda t: data.e2i[(
        f'{URI_PREFIX}entity#binning{t}#2', f'{URI_PREFIX}datatype#bin')])
    object_mapping22 = np.vectorize(lambda t: num_bins-1 if t>=num_bins else t)

    predicat_mapping2 = np.vectorize(
        lambda t: data.r2i[f'{URI_PREFIX}predicat#binning{relation}#2'])

    sub_df2[:, 1] = torch.tensor([predicat_mapping2(sub_df2[:, 2])], dtype=torch.int32)
    sub_df2[:, 2] = torch.tensor([object_mapping22(sub_df2[:, 2])], dtype=torch.int32)
    sub_df2[:, 2] = torch.tensor([object_mapping2(sub_df2[:, 2])], dtype=torch.int32)
    data.triples = torch.cat((data.triples, sub_df2), 0)


# Subpopulation 

In [4]:
from preprocess import subpopulation_binning
data = subpopulation_binning(data, 10)

j: 0 - prop: 0.7784313725490196 - kl_div: 100.03431
j: 1 - prop: 0.6215686274509804 - kl_div: 153.5318
j: 2 - prop: 1.0 - kl_div: 0.0
j: 3 - prop: 1.0 - kl_div: 0.0
j: 4 - prop: 1.0 - kl_div: 0.0
j: 5 - prop: 1.0 - kl_div: 0.0
j: 6 - prop: 1.0 - kl_div: 0.0
j: 7 - prop: 1.0 - kl_div: 0.0
j: 8 - prop: 1.0 - kl_div: 0.0
j: 9 - prop: 1.0 - kl_div: 0.0
j: 10 - prop: 1.0 - kl_div: 0.0
j: 11 - prop: 1.0 - kl_div: 0.0
j: 12 - prop: 1.0 - kl_div: 0.0
j: 13 - prop: 1.0 - kl_div: 0.0
j: 14 - prop: 1.0 - kl_div: 0.0
j: 15 - prop: 0.7764705882352941 - kl_div: 100.63174
j: 16 - prop: 0.06274509803921569 - kl_div: 92.89656
j: 17 - prop: 0.00392156862745098 - kl_div: 14.92516
j: 0 - prop: 1.0 - kl_div: 0.0
j: 1 - prop: 1.0 - kl_div: 0.0
j: 2 - prop: 1.0 - kl_div: 0.0
j: 3 - prop: 1.0 - kl_div: 0.0
j: 4 - prop: 1.0 - kl_div: 0.0
j: 5 - prop: 1.0 - kl_div: 0.0
j: 6 - prop: 1.0 - kl_div: 0.0
j: 7 - prop: 1.0 - kl_div: 0.0
j: 8 - prop: 1.0 - kl_div: 0.0
j: 9 - prop: 1.0 - kl_div: 0.0
j: 10 - prop: 1.0 - 

In [6]:
data.triples

AttributeError: 'NoneType' object has no attribute 'triples'